In [ ]:
import numpy as np
from adaptive_latents.input_sources import LDS
from adaptive_latents import ArrayWithTime
import adaptive_latents
from adaptive_latents import VJF
from tqdm.notebook import trange, tqdm

import matplotlib.pyplot as plt


In [ ]:
rng = np.random.default_rng(0)

In [ ]:
transitions_per_rotation = 30
radius = 10
_, Y, _ = LDS.run_nest_dynamical_system(rotations=1000, transitions_per_rotation=transitions_per_rotation, radius=radius, u_function=lambda **_: np.zeros(3), rng=rng)
Y = Y

predictor = VJF(latent_d=2, rng=rng)

predictor.offline_run_on([(Y, 'X')], convinient_return=False, show_tqdm=True);


In [ ]:
trajectory = []
for i in trange(0, transitions_per_rotation+1):
    stream = 'pred_obs_space'  # TODO: change this string
    prediction = predictor.partial_fit_transform(ArrayWithTime([[i * Y.dt]], Y.t[-1]), stream=stream)
    trajectory.append(prediction)

trajectory = np.squeeze(trajectory)


In [ ]:
%matplotlib qt
plt.plot(Y[:, 0], Y[:, 1])
plt.plot([Y[-1, 0], trajectory[0, 0]], [Y[-1, 1], trajectory[0, 1]], '--.', color='C2')
plt.plot(trajectory[:, 0], trajectory[:, 1], '.-')
plt.axis('equal')
plt.show()


In [ ]:
predictor.show_nstep_cloud(30)